In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from Plotter.ConfigMatplotlib import ConfigMatplotlib
from Plotter.ConfigHist  import ConfigHist
from Common.CommonHelper import CommonHelper
from Common.StackList    import StackList
from Common.IO           import openJson
from Samples.ConfigData  import ConfigData

In [1]:
class HistoSample( StackList, ConfigMatplotlib, ConfigHist, ConfigData):
    def __init__(self,stack = None,name=None,Print=False):
        StackList.__init__(self, name=name, stack=stack ,Print=Print)

        if name is not None:
            if type(name) is list:
                self.names     = name
            else:
                self.names     = [ name ]                
    
    def size(self,sample=None,variable=None,weighted=True):
        if sample is None:
            return len(self)
        else:
            try:
                return self[sample].size(variable,weighted)
            except:
                return self[sample].size(weighted)
            
    def sizes(self,variable = None, weighted=True): 
        return [ ist.size(variable,weighted) for ist in self ]
    
    def append(self,stack,name=None):
        if name is None and stack.name is None:
            raise "Name is not specified, provide argument \"name\" or setit in Histo"
        
        if name is None:
            name = stack.name
        else:
            stack.name = name
        
        super().append(stack,stack.name)
                
    def getEntries(self):
        return self.sizes(weighted=False)
    
    def _setOrder(self,ind):
        self.names = list(np.array(self.names)[list(ind)])
        self.stack = self[list(ind)]
    
    def _order(self,order="l2m",ind=None,weighted=True,):
        if ind is None:
            ind = np.argsort(self.sizes())
            if order == "l2m":
                self._setOrder(ind)
            elif order == "m2l":
                self._setOrder(ind[::-1])
            else:
                print("Type of ordering is not supported")
        else:
            try: 
                ind  = [self.names.index(nm)  for nm in ind]
            except:
                ind = ind
                
            self._setOrder(ind)
    
    def Name2Index(self,sample):
        index = np.arange(len(self))
        try:
            #return int(index[np.array(self.name) == sample])
            return int(index[np.array(self.names) == sample])
        except:
            return None
        
    def getData(self):
        return self[self.getDataName()]
    
    def getDataName(self):
        names = []
        for nm in self.names:
            if super().isData(nm):
                return str(nm)
            
        return None
        
    def getMC(self):
        return self[self.getMCName()]
        
    def getMCName(self):
        names = []
        for nm in self.names:
            if not super().isData(nm):
                names.append(str(nm))
        return names
    
    def getProperties(self,names=None,color=None,label=None,histtype=None):
        if names is None:
            names = self.names
            
        prop = {}
        if color is not None:
            prop['color'] = color
        else:
            prop['color']    = [ super(HistoSample,self).getColor(name) for name in names]
        
        if label is not None:
            prop['label'] = label
        else:
            prop['label']    = [ super(HistoSample,self).getLabel(name) for name in names]
        
        if histtype is not None:
            prop['histtype'] = histtype
        else:
            prop['histtype'] = super(HistoSample,self).getHisttpe(names[0])

        return prop
    
    def getMCCounts(self,hist):
        return hist[-1]
    
    def pop(self,i):
        self.names.pop(i)
        return super().pop(i)
        
    def merge(self,samples,name=None):
        if name is None:
            raise BaseException("Provide a name for the merged sample")

        Merge = sum([self.pop(self.Name2Index(smp)) for smp in samples if CommonHelper.Type.isNumeric(self.Name2Index(smp))])

        self.append(Merge,name)
        
    def savehists(self,path,prefix=""):
        for i, histo in enumerate(self):            
            histo.save(path,prefix)

    def savefig(self,fig,fullpath):
        fig.savefig(fullpath)
        
    def savefigs(self,path,Type="single",ind=None):
        for var in self[0].variable:
            for log in [True,False]:
                fig, _ = self.plot(var,log=log,ind=ind)
                LOG  = 'log/' if log else 'linear/'
                self.savefig(fig,fullpath=os.path.join(path,LOG,Type,var) )
        
            
    def CollaborationTitle(self,ax,era):
        param = {
                #'fontfamily':'Palatino',
                'fontweight':'bold',
                'size':20
                }
        ax.text(0,1.01,"CMS",
                 **param,
                 transform=ax.transAxes
                 )
        param = {
                #'fontweight':'bold',
                'size':20
                }
        ax.text(0.1,1.01,
                 #"work in progress",
                "Preliminary",
                 **param,
                 transform=ax.transAxes
                 )


        param = {
                'fontweight':'bold',
                'size':18
                }

        path = '/home/jcordero/CMS/SMP_ZGamma/json/data/'
        lumi = str(round(float(openJson(path+'lumi.json')[era]),1))
        ax.text(0.57,1.01, r""+lumi+" fb$^{-1}$ (13 TeV) "+era+"",
                 **param,
                 transform=ax.transAxes
                 )
    
    
                
            
    def plotDataMC(self,bins,Data,MC,ax=None):
        if ax is None:
            fig = plt.figure()
            ax = plt.gca()
        Data,MC = np.array(Data), np.array(MC)
            
        Ratio = Data/MC
        binc = CommonHelper.Plot.BinFormat(Bins=bins, Type="center")
        err = Data/MC*(np.sqrt(1/Data + 1/MC))
        
        prop = {'marker':'o',
                'color':'k',
                'linestyle':'',
                'linewidth':1.5,
               }
        
        plt.errorbar(binc, Ratio,
                     xerr = np.diff(bins)/2,
                     yerr = err,
                     **prop,
                    )
        ax = plt.gca()
        ax.set_ylim([0.5,1.5])
        
        return ax
    
    def getStatUncertainty(self,x,y):
        yy = []
        for i,h in enumerate(y):
            yy.append(h)
            yy.append(h)
        yy = np.array(yy)    

        xx = []
        for i,h in enumerate(x):
            if i == 0 or i == len(x)-1:
                xx.append(h)
            else:
                xx.append(h)
                xx.append(h)
        
        return np.array(xx),np.array(yy),np.sqrt(yy)
        
    
    def plot(self,variable,log=False,
             order="l2m",ind=None,
             shape = (4,1), loc   = 0,
             xranges = None,yranges = None,limranges=None,
             WithYield = None,
             Type = "single",Debug=False,
             Create = True,
            ):
        super().setRC(plt.rc,Type=Type)
        
        if Create:
            fig = plt.figure()
            
        plt.subplot2grid(shape,(0,loc),rowspan = 3, colspan = 1)
        
        self._order(order,ind=ind)
        
        mc = [ self[name][variable].values  for name in self.getMCName() ]
        bins = self.stack[0][variable].bins
        binc = CommonHelper.Plot.BinFormat(Bins=bins, Type="center")
        binc = [binc]*len(mc)
        
        prop = self.getProperties(names = self.getMCName() )
        if WithYield:
            for i,label in enumerate(prop['label']):
                prop['label'][i] += ": "+ str(round(np.sum(mc[i])))
        mchist = plt.hist(
                            binc,
                            bins = bins,
                            weights  = mc,
                            stacked = True,
                            **prop,
                            )
        ax = plt.gca()
        
        statx, statyc, statysig = self.getStatUncertainty(mchist[1],mchist[0][-1])
        ax.fill_between(statx,statyc+statysig, statyc-statysig,
                        color='grey',alpha = 0.05,
                        zorder = 5)
        ax.fill_between(statx,statyc+statysig, statyc-statysig,
                        facecolor='none',alpha = 0.5,edgecolor='black',
                        hatch = '////',linewidth=0.0,zorder=6,
                        label = "MC stat unc")
        
        self.CollaborationTitle(ax,self.name)

        ### Plot Data
        if self.getDataName() is not None:
            data = self[self.getDataName()][variable].values
            err = np.sqrt(data)
            
            prop = {'marker':'o',
                    'color':'k',    
                    'linestyle':'',
                    'label':'Data',
                   }
            
            if WithYield:
                prop['label'] += ": "+str(round(np.sum(data)))
            plt.errorbar(binc[0],
                         data,
                         xerr = np.diff(bins)/2,
                         yerr = err,
                         **prop
                        )

            ax = plt.gca()          
            #ax.set_xticklabels([])
            
            ### Plot Data/MC
            plt.subplot2grid(shape,(3,loc), rowspan = 1, colspan = 1, sharex = ax)
            ax1 = plt.gca()            

            self.plotDataMC(bins,data,mchist[0][-1],ax1)
                        
            #ax1.set_xticklabels(bins)
            ax1.set_xlabel(variable)
            ax1.hlines(1,xmin=bins[0],xmax=bins[-1],linestyles='--',colors='r',alpha=0.5)
            
            plt.tight_layout()
            
            fig = plt.gcf()
            fig.subplots_adjust(hspace=0)
        else:
            ax.set_xlabel(variable)
            
        ax.set_ylabel('Counts')
        ax.legend()
        
        if xranges is not None:
            ax.set_xlim(xranges)
            if self.getDataName() is not None:
                ax1.set_xlim(xranges)
        
        if yranges is not None:
            ax.set_ylim(yranges)
            
        if limranges is not None:
            ax1.set_ylim(limranges)
        
        if log:
            ax.set_yscale('log')
            
        if Debug:
            fig = plt.gcf()
            try:
                return fig,[ax,ax1],binc,mc,data,prop,
            except:
                return fig,ax,binc,mc,data,prop,
        else:
            fig = plt.gcf()
            try:
                return fig,[ax,ax1]
            except:
                return fig,ax

NameError: name 'StackList' is not defined